# Aggregates and Groups

## Setup

Import our modules again:

In [1]:
import numpy as np
import pandas as pd

And load the MovieLens data.  We're going to pass the `memory_use='deep'` to `info`, so we can see the total memory use including the strings.

In [2]:
# read the movies table
file_path = "./ml-latest-small"
movies = pd.read_csv('{0}/movies.csv'.format(file_path))
movies.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 1.5 MB


In [3]:
# read the rating table
ratings = pd.read_csv('{0}/ratings.csv'.format(file_path))
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


Quickly preview the `ratings` frame:

In [4]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


## Aggregate Functions

An **aggregate function** combines a series (or array) into a single value:

In [5]:
ratings['rating'].mean()

3.501556983616962

In [6]:
ratings['rating'].sum()

353083.0

Alternate form --- function from `numpy`:

In [7]:
np.sum(ratings['rating'])

353083.0

## Series Size

How big is the series?

In [8]:
ratings['rating'].size

100836

In [9]:
len(ratings['rating'])

100836

We can also get the *array shape*:

In [10]:
ratings['rating'].shape

(100836,)

And the count of non-null values (we don't have any null values!):

In [11]:
ratings['rating'].count()

100836

- `.shape` returns the array shape as a tuple — the array is one-dimensional with length 25M
- `.size` returns the series size (length). `len(…)` is identical.
- `.count()` counts values, **not** including missing values

## Quantiles

Let's see the `quantile` function:

In [12]:
ratings['rating'].quantile(0.5)

3.5

In [13]:
ratings['rating'].quantile(0.2)

3.0

Fascinating! 80% of ratings are 3.0 or higher.

## Grouped Aggregates

We can group by a column and compute aggregates within the group.

How many ratings per movie?

In [14]:
ratings.groupby('movieId')['rating'].count()

movieId
1         215
2         110
3          52
4           7
5          49
         ... 
193581      1
193583      1
193585      1
193587      1
193609      1
Name: rating, Length: 9724, dtype: int64

We can compute multiple aggregates at the same time:

In [15]:
movie_stats = ratings.groupby('movieId')['rating'].agg(['mean', 'count'])
movie_stats

,mean,count
movieId,,
1,3.920930,215
2,3.431818,110
3,3.259615,52
4,2.357143,7
5,3.071429,49
...,...,...
193581,4.000000,1
193583,3.500000,1
193585,3.500000,1


This result is a **data frame**, again indexed by `movieId`.

## Finding Largest

We can get the 10 movies with the most ratings:

In [16]:
movie_stats.nlargest(10, 'count')

,mean,count
movieId,,
356,4.164134,329
318,4.429022,317
296,4.197068,307
593,4.161290,279
2571,4.192446,278
260,4.231076,251
480,3.750000,238
110,4.031646,237
589,3.970982,224


## Joining

We want to combine our stats with movie info.

* `on='movieId'` says to use the `movieId` column of the first frame instead of its index.
* Matches values to index in other frame

In [17]:
movie_info = movies.join(movie_stats, on='movieId')
movie_info

,movieId,title,genres,mean,count
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930,215.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818,110.0
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615,52.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143,7.0
4,5,Father of the Bride Part II (1995),Comedy,3.071429,49.0
...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,4.000000,1.0
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,3.500000,1.0
9739,193585,Flint (2017),Drama,3.500000,1.0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,3.500000,1.0


In [18]:
movie_info.nlargest(10, 'count')

,movieId,title,genres,mean,count
314,356,Forrest Gump (1994),Comedy|Drama|Romance|War,4.164134,329.0
277,318,"Shawshank Redemption, The (1994)",Crime|Drama,4.429022,317.0
257,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,4.197068,307.0
510,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,4.161290,279.0
1939,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,4.192446,278.0
224,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,4.231076,251.0
418,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,3.750000,238.0
97,110,Braveheart (1995),Action|Drama|War,4.031646,237.0
507,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,3.970982,224.0
461,527,Schindler's List (1993),Drama|War,4.225000,220.0


In [19]:
movie_info['count'].describe()

count    9724.000000
mean       10.369807
std        22.401005
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       329.000000
Name: count, dtype: float64